# Heat days in 2050

Projected number of extreme heat days, where temperature exceeds the top 1% of historic values, in the year 2050. This map uses the RCP 8.5 emissions scenario from the 4th National Climate Assessment. [Esri map](https://noaa.maps.arcgis.com/apps/mapviewer/index.html?webmap=196b9785f5f745358daafcf33df4f072) | [Data](https://services8.arcgis.com/O5dJJKQg5b6RjO3d/ArcGIS/rest/services/LOCA_Counties_RCP8_2050/FeatureServer/0)


#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np
from datetime import date
import urllib
import json

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 10000
pd.options.display.max_rows = 10000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

---

## Read data

#### Query the [Esri service](https://services8.arcgis.com/O5dJJKQg5b6RjO3d/ArcGIS/rest/services/LOCA_Counties_RCP8_2050/FeatureServer/0) for just its features despite the 2,000 record limit | Data summary [here](https://noaa.maps.arcgis.com/home/item.html?id=ce694649f803487caeee3cb6833e769c&view=list&sortOrder=desc&sortField=defaultFSOrder)

 `where: 1=1`  `out fields: *` `result offset:2000`

In [6]:
data_list = []

for o in ["0", "2000"]:
    url = f"https://services8.arcgis.com/O5dJJKQg5b6RjO3d/ArcGIS/rest/services/LOCA_Counties_RCP8_2050/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=false&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset={o}&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="
    r = urllib.request.urlopen(url)
    data_list.append(
        json.loads(r.read().decode(r.info().get_param("charset") or "utf-8"))
    )

In [7]:
dataframes = []

for d in data_list:
    src = pd.DataFrame(d["features"])
    dataframes.append(pd.json_normalize(src["attributes"]))

In [8]:
df = pd.concat(dataframes)

In [9]:
df.columns = df.columns.str.lower()

In [10]:
df.columns.to_list()

['objectid',
 'name',
 'state_name',
 'state_fips',
 'cnty_fips',
 'fips',
 'sqmi',
 'tmin',
 'tminmax5day',
 'tmindaysbelow28f',
 'tmindaysabove99th',
 'tmindaysabove90f',
 'tmindaysabove85f',
 'tmindaysabove80f',
 'tmindaysabove75f',
 'tmin32f',
 'tmin5day',
 'tmin1day',
 'tmax',
 'tmaxdaysbelow1st',
 'tmaxdaysabove99th',
 'tmax115f',
 'tmax110f',
 'tmax105f',
 'tmax100f',
 'tmax95f',
 'tmax90f',
 'tmax86f',
 'tmax32f',
 'tmax5day',
 'tmax1day',
 'tavg',
 'prmax5day',
 'prmax1day',
 'prdaysabove99th',
 'pr4in',
 'pr3in',
 'pr2in',
 'pr1in',
 'lastfreezedate',
 'hdd',
 'growingseason',
 'growingseason41f',
 'growingseason28f',
 'gdd',
 'gddmod',
 'firstfreezedate',
 'consecwd',
 'consecdd',
 'consecddjja',
 'cdd',
 'tminmax5dayhistorical',
 'tminhistorical',
 'tmindaysbelow28fhistorical',
 'tmindaysbelow1sthistorical',
 'tmindaysabove99thhistorical',
 'tmindaysabove90fhistorical',
 'tmindaysabove85fhistorical',
 'tmindaysabove80fhistorical',
 'tmindaysabove75fhistorical',
 'tmin32fhis

In [11]:
df_slim = df[
    [
        "name",
        "state_name",
        "state_fips",
        "cnty_fips",
        "fips",
        "tmax",
        "tmaxhistorical",
        "tmaxdaysabove99th",
        "tmaxdaysabove99thhistorical",
        "tmax100f",
        "tmax100fhistorical",
        "tmax95f",
        "tmax95fhistorical",
        "tavg",
        "tavghistorical",
    ]
].copy()

In [12]:
df_slim.sort_values("tmax100fhistorical", ascending=False).head()

,name,state_name,state_fips,cnty_fips,fips,tmax,tmaxhistorical,tmaxdaysabove99th,tmaxdaysabove99thhistorical,tmax100f,tmax100fhistorical,tmax95f,tmax95fhistorical,tavg,tavghistorical
169,Imperial,California,06,025,06025,4.439210,87.356323,22.767766,3.670000,32.188213,102.427010,29.443746,136.710343,4.401649,72.261615
81,Yuma,Arizona,04,027,04027,4.769266,86.804152,25.879237,3.669972,37.676921,98.026017,32.398136,134.992345,4.725932,71.768023
73,La Paz,Arizona,04,012,04012,4.734620,85.484455,23.769835,3.670000,37.136436,94.595743,31.198185,130.140132,4.637261,70.413003
74,Maricopa,Arizona,04,013,04013,4.796359,85.420453,26.988591,3.670000,40.613708,89.816980,33.656292,128.008960,4.665906,69.807181
78,Pinal,Arizona,04,021,04021,4.765723,83.373274,26.824071,3.669971,43.346224,68.638643,38.208319,110.109852,4.586962,67.933392


In [13]:
df_slim["pred_tmax"] = df_slim["tmaxhistorical"] + df_slim["tmax"]
df_slim["pred_days_99th"] = (
    df_slim["tmaxdaysabove99thhistorical"] + df_slim["tmaxdaysabove99th"]
)
df_slim["pred_tmax_100f"] = df_slim["tmax100fhistorical"] + df_slim["tmax100f"]
df_slim["pred_tmax_95f"] = df_slim["tmax95fhistorical"] + df_slim["tmax95f"]
df_slim["pred_tavg"] = df_slim["tavghistorical"] + df_slim["tavg"]

In [14]:
temps = (
    df_slim[
        [
            "name",
            "state_name",
            "state_fips",
            "cnty_fips",
            "fips",
            "tmaxhistorical",
            "tmax",
            "pred_tmax",
            # Historically (1976-2005) the area experienced
            "tmaxdaysabove99thhistorical",
            # Number of days per year warmer than the top 1% historically
            "pred_days_99th",
            "tavghistorical",
            "pred_tavg",
            "tmax95fhistorical",
            "pred_tmax_95f",
        ]
    ]
    .sort_values("pred_tmax_95f", ascending=False)
    .copy()
)

In [15]:
temps.head()

,name,state_name,state_fips,cnty_fips,fips,tmaxhistorical,tmax,pred_tmax,tmaxdaysabove99thhistorical,pred_days_99th,tavghistorical,pred_tavg,tmax95fhistorical,pred_tmax_95f
81,Yuma,Arizona,04,027,04027,86.804152,4.769266,91.573418,3.669972,29.549209,71.768023,76.493955,134.992345,167.390480
169,Imperial,California,06,025,06025,87.356323,4.439210,91.795533,3.670000,26.437766,72.261615,76.663265,136.710343,166.154089
74,Maricopa,Arizona,04,013,04013,85.420453,4.796359,90.216812,3.670000,30.658591,69.807181,74.473087,128.008960,161.665252
73,La Paz,Arizona,04,012,04012,85.484455,4.734620,90.219076,3.670000,27.439835,70.413003,75.050264,130.140132,161.338317
741,Zapata,Texas,48,505,48505,85.430476,4.780000,90.210476,3.670000,43.724286,73.617143,78.221270,106.046508,157.977778


---

## Geography 

In [16]:
geo_src = gpd.read_file("data/raw/usa_counties_esri_simple_mainland.json")[
    ["fips", "population", "households", "crop_acr17", "geometry"]
]

In [17]:
geo_src.head()

,fips,population,households,crop_acr17,geometry
0,01001,58224,20221,36890,"POLYGON ((-86.41312 32.70739, -86.71422 32.705..."
1,01003,227660,73180,110438,"MULTIPOLYGON (((-87.56491 30.28162, -87.56860 ..."
2,01005,26326,9820,37304,"POLYGON ((-85.05603 32.06306, -85.10498 32.062..."
3,01007,23066,7953,15823,"POLYGON ((-87.06574 33.24691, -87.07665 33.246..."
4,01009,59970,21578,43793,"POLYGON ((-86.45302 34.25932, -86.48687 34.260..."


---

## Merge 'em

In [18]:
geo_df = gpd.GeoDataFrame(pd.merge(temps, geo_src, on=["fips"]))

In [19]:
geo_df.head()

,name,state_name,state_fips,cnty_fips,fips,tmaxhistorical,tmax,pred_tmax,tmaxdaysabove99thhistorical,pred_days_99th,tavghistorical,pred_tavg,tmax95fhistorical,pred_tmax_95f,population,households,crop_acr17,geometry
0,Yuma,Arizona,04,027,04027,86.804152,4.769266,91.573418,3.669972,29.549209,71.768023,76.493955,134.992345,167.390480,214176,64767,234278,"POLYGON ((-113.33506 33.37747, -113.95750 33.3..."
1,Imperial,California,06,025,06025,87.356323,4.439210,91.795533,3.670000,26.437766,72.261615,76.663265,136.710343,166.154089,179185,49126,504037,"POLYGON ((-114.62714 33.43356, -114.83088 33.4..."
2,Maricopa,Arizona,04,013,04013,85.420453,4.796359,90.216812,3.670000,30.658591,69.807181,74.473087,128.008960,161.665252,4477918,1411583,257187,"POLYGON ((-111.49500 33.99996, -111.72634 34.0..."
3,La Paz,Arizona,04,012,04012,85.484455,4.734620,90.219076,3.670000,27.439835,70.413003,75.050264,130.140132,161.338317,21029,9198,102644,"POLYGON ((-113.33343 34.31792, -113.34323 34.3..."
4,Zapata,Texas,48,505,48505,85.430476,4.780000,90.210476,3.670000,43.724286,73.617143,78.221270,106.046508,157.977778,15006,4297,18856,"POLYGON ((-98.95468 27.26940, -99.33362 27.273..."


In [20]:
geo_df["population"].sum()

331595501

#### How many counties will have more than one 95F day in 2050? 

In [21]:
len(geo_df[geo_df["pred_tmax_95f"] >= 1])

3060

In [22]:
future = geo_df[geo_df["pred_tmax_95f"] >= 1]["households"].sum()

#### How many counties have historically had more than one 95F day? 

In [23]:
len(geo_df[geo_df["tmax95fhistorical"] >= 1])

2578

In [24]:
historic = geo_df[geo_df["tmax95fhistorical"] >= 1]["households"].sum()

In [25]:
future - historic

17182155

#### How many places have or will have more than 50? 

In [26]:
len(geo_df[geo_df["tmax95fhistorical"] >= 50])

153

In [27]:
geo_df[geo_df["tmax95fhistorical"] >= 50]["population"].sum()

30574769

#### How many places will have more than 50? 

In [28]:
len(geo_df[geo_df["pred_tmax_95f"] >= 50])

1099

In [29]:
geo_df[geo_df["pred_tmax_95f"] >= 50]["population"].sum()

109236138

---

## Export

In [30]:
geo_df.to_file("data/processed/counties_extreme_heat_2020.geojson", driver="GeoJSON")